# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import sys
import json
import pandas as pd
import numpy as np
from random import random
from operator import add
from pyspark.sql import SparkSession
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
SparkSession

numpy version: 1.18.4
pandas version: 1.0.3


pyspark.sql.session.SparkSession

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%2<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=exact(random()/pow(2,31)-0.5)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_spark_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_spark_model")
print(df.describe())
print(param)

           label  feature_1  feature_0
count  10.000000  10.000000  10.000000
mean    0.500000   0.100146   0.206111
std     0.527046   0.623452   0.616813
min     0.000000  -0.894577  -0.952523
25%     0.000000  -0.328584  -0.049762
50%     0.500000   0.239535   0.269672
75%     1.000000   0.554405   0.633472
max     1.000000   0.819362   0.980341
{'options': {'params': {'mode': 'stage', 'algo': 'barebone'}, 'args': ['label', 'feature_*'], 'target_variable': ['label'], 'feature_variables': ['feature_*'], 'model_name': 'barebone_spark_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '1000', 'max_distinct_cat_values_for_classifiers': '1000', 'max_distinct_cat_values_for_scoring': '1000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': [

In [5]:
print(param['options']['model_name'])

barebone_spark_model


## Stage 2 - create and initialize a model

In [6]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    appName = "barebone_spark_model"
    if 'options' in param:
        if 'model_name' in param['options']: 
            appName = param['options']['model_name']
    spark = SparkSession\
        .builder\
        .appName(appName)\
        .getOrCreate()
    model['spark'] = spark
    return model

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'spark': <pyspark.sql.session.SparkSession object at 0x7f3394657f90>}


In [8]:
model['spark']

In [9]:
model['spark'].sparkContext.getConf().getAll()

[('spark.app.name', 'barebone_spark_model'),
 ('spark.driver.port', '45801'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1590659988876'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '7e6865cc4818'),
 ('spark.ui.showConsoleProgress', 'true')]

## Stage 3 - fit the model

In [10]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [11]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [12]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    spark = model['spark']
    partitions = 4
    sdf = spark.createDataFrame(df)
    rdd = sdf.rdd.map(lambda row: 1 if row["feature_0"] ** 2 + row["feature_1"] ** 2 <= 1 else 0)
    result = pd.DataFrame(rdd.collect())
    return result

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

   0
0  0
1  1
2  1
3  0
4  0
5  1
6  1
7  1
8  0
9  1


In [19]:
spark = model['spark']
sdf = spark.createDataFrame(df)
rdd = sdf.rdd.map(lambda row: 1 if row["feature_0"] ** 2 + row["feature_1"] ** 2 <= 1 else 0)
result = [{"result":row} for row in rdd.collect()]
result

[{'result': '0'},
 {'result': '1'},
 {'result': '1'},
 {'result': '0'},
 {'result': '0'},
 {'result': '1'},
 {'result': '1'},
 {'result': '1'},
 {'result': '0'},
 {'result': '1'}]

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
#    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
#        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
#    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
#        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"spark": "no model"}
    if model:
        returns = {"spark_info": str(model['spark'].sparkContext.getConf().getAll()) }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code